## 2.6. 디코더

인코더와 디코더의 차이는 인코더는 **멀티헤드 어텐션**을 사용하지만 디코더는 **마스크 멀티 헤드 어텐션**을 사용한다. 디코더는 생성을 담당한느 부분으로, 사람이 글을 쓸 때 앞 단어부터 순차적으로 작성하는 것처럼 트랜스포머 모델도 앞에서 생성한 토큰을 기반으로 다음 토큰을 생성한다.

디코더가 텍스트를 생성할 때 디코더는 이전까지 생성한 텍스트만 확인할 수 있다. 하지만 학습할 때 인코더, 디코더 모두 전체 문장을 학습한다. 따라서 디코더가 전체 문장을 확인하게 되는 문제가 생긴다. 이를 해결하기 위해 **마스크 멀티 헤드 어텐션**을 사용한다.

아래 코드에서 is_causal=True로 설정하면, 마스크 멀티 헤드 어텐션을 사용할 수 있다. torch.ones로 모두 1인 행렬에 tril 함수를 적용해 대각선 아래 부분만 1로 유지하고 나머지는 -inf로 마스크를 생성한다. 여기에 어텐션 스코어를 곱해 마스킹한다.

In [2]:
# 예제 2.13 디코더에서 어텐션 연산 ( 마스크 어텐션 )

import math
import torch
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1) # 키의 차원
    scores = querys @ keys.transpose(-2, -1) / math.sqrt(dim_k) # 점수 계산
    if is_causal:
        query_length = querys.size(2)
        key_length = keys.size(2)
        temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0) # 마스크 생성
        scores.masked_fill_(temp_mask, float('-inf')) # 마스크 적용
        weights = F.softmax(scores, dim=-1) # 소프트맥스 함수 적용
        return weights @ values # 가중합 계산

**크로스 어텐션**

크로스 어텐션은 인코더와 디코더를 연결하는 어텐션이다. 

In [3]:
# 예제 2.14 크로스 어텐션이 포함된 디코더 층
import torch.nn as nn

class PreLayerNormFeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout):
        super().__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward) # 1번째 선형 층
        self.linear2 = nn.Linear(dim_feedforward, d_model) # 2번째 선형 층
        self.dropout1 = nn.Dropout(dropout) # 드롭아웃 1
        self.dropout2 = nn.Dropout(dropout) # 드롭아웃 2
        self.activation = nn.GELU() # 활성화 함수
        self.norm = nn.LayerNorm(d_model) # 층 정규화

    def forward(self, src):
        x = self.norm(src)
        x = x + self.linear2(self.activation(self.linear1(x))) # 2개의 선형 층과 활성화 함수
        x = self.dropout2(x) # 드롭아웃 2
        return x

class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, n_head, dim_feedforward=2048, dropout=0.1):
        super().__init__()

        self.self_attn = nn.MultiheadAttention(d_model, d_model, n_head)
        self.multihead_attn = nn.MultiheadAttention(d_model, d_model, n_head)
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, tgt, encoder_output, is_causal=True):
        # 셀프 어텐션 연산
        x = self.norm1(tgt)
        x = x + self.dropout1(self.self_attn(x,x,x))

        # 크로스 어텐션 연산
        x = self.norm2(x)
        x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output, is_causal=is_causal))

        # 피드 포워드 연산
        return self.feed_forward(x)

        

In [2]:
# 예제 2.15 디코더 구현
import copy
import torch.nn as nn
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerDecoder(nn.Module):
    def __init__(self, decoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers

    def forward(self, tgt, src):
        output = tgt
        for mod in self.layers:
            output = mod(output, src)
        return output